In [1]:
import cvxpy as cp    ## To install cvxpy, https://www.cvxpy.org/install/index.html
import numpy as np

from scipy.linalg import sqrtm

## 1. State Preparation

$$
q_0 = 1/3, |\psi_0\rangle = |0\rangle\\
q_1 = 1/3, |\psi_1\rangle = |1\rangle\\
q_2 = 1/3, |\psi_2\rangle = |+\rangle
$$

In [2]:
q_list = [1/3] * 3
rho_0 = np.array([[1,0],[0,0]])
rho_1 = np.array([[0,0],[0,1]])
rho_2 = np.array([[1/2,1/2],[1/2,1/2]])
rho_list = [rho_0, rho_1,rho_2]

## 2. Semi-Definite Programming for MED

### Primal Problem

$$
\max \sum_{i=0}^{l-1}q_i\text{Tr}[E_i\rho_i]\\
\text{Subject to } E_i \ge 0, \forall i=1,\cdots, l \\
\sum^{l-1}_{i=0}E_i=I
$$

In [10]:
# Create l 2x2 matrix variables
E = [cp.Variable((2,2), hermitian=True) for x in range(3)]

# Create constraints
constraints = [E[0] + E[1] + E[2] == np.eye(2)]
constraints += [
    E[i] >> 0 for i in range(3)
]

# Form objective.
q_rho = [q_list[i] * rho_list[i] for i in range(3)]
obj = cp.real(cp.trace(E[0] @ q_rho[0]) + cp.trace(E[1] @ q_rho[1]) + cp.trace(E[2] @ q_rho[2]))

prob = cp.Problem(cp.Maximize(obj), constraints)
prob.solve()

print("status:", prob.status)
print("optimal value", prob.value)
print("A solution E is")
for i in range(3):
    print("E_"+str(i)+" =", E[i].value)


WARN: A->p (column pointers) not strictly increasing, column 9 empty
WARN: A->p (column pointers) not strictly increasing, column 12 empty
WARN: A->p (column pointers) not strictly increasing, column 13 empty
WARN: A->p (column pointers) not strictly increasing, column 16 empty
WARN: A->p (column pointers) not strictly increasing, column 17 empty
WARN: A->p (column pointers) not strictly increasing, column 20 empty
status: optimal
optimal value 0.666666642285187
A solution E is
E_0 = [[9.99999967e-01+0.j 4.20319381e-09+0.j]
 [4.20319381e-09+0.j 3.12216184e-08+0.j]]
E_1 = [[3.12216155e-08+0.j 4.20319381e-09+0.j]
 [4.20319381e-09+0.j 9.99999967e-01+0.j]]
E_2 = [[ 2.01313661e-09+0.j -8.45401352e-09+0.j]
 [-8.45401352e-09+0.j  2.01313383e-09+0.j]]


### Dual Problem

$$
\min_{K} \text{Tr}[K]\\
\text{Subject to } K - q_i\rho_i \ge 0, \forall i=1,\cdots, l
$$

In [5]:
K = cp.Variable((2,2), hermitian=True)
constraints = [K - q_rho[i] >> 0 for i in range(3)]
prob = cp.Problem(cp.Minimize(cp.real(cp.trace(K))), constraints)
prob.solve()
print("status:", prob.status)
print("optimal value", prob.value)
print("A solution K is")
print(K.value)

WARN: A->p (column pointers) not strictly increasing, column 3 empty
WARN: A->p (column pointers) not strictly increasing, column 6 empty
status: optimal
optimal value 0.6666643166227835
A solution K is
[[3.33332158e-01+0.j 3.77977106e-06+0.j]
 [3.77977106e-06+0.j 3.33332158e-01+0.j]]


## 3. Pretty Good Measurement

$$
\Pi_i = q_i \rho^{-1/2} \rho_i \rho^{-1/2} \text{, where } \rho = \sum_i q_i\rho_i
$$

In this example, $\rho = \frac{1}{3}\left(|0\rangle\langle 0| + |1\rangle\langle 1| + |+\rangle\langle +| \right)$. By the spectral decomposition,
$$
\rho = \begin{pmatrix}
\frac{1}{\sqrt{2}} & \frac{1}{\sqrt{2}} \\ 
\frac{1}{\sqrt{2}} & -\frac{1}{\sqrt{2}}
\end{pmatrix}
\begin{pmatrix}
\frac{2}{3} & 0 \\ 
0 & \frac{1}{3}
\end{pmatrix}
\begin{pmatrix}
\frac{1}{\sqrt{2}} & \frac{1}{\sqrt{2}} \\ 
\frac{1}{\sqrt{2}} & -\frac{1}{\sqrt{2}}
\end{pmatrix}
$$

In [18]:
rho_cvsum = np.array([[1/2,1/6],[1/6,1/2]])
rho_cvsum

array([[0.5       , 0.16666667],
       [0.16666667, 0.5       ]])

$$
\rho^{1/2} = \begin{pmatrix}
\frac{1}{\sqrt{2}} & \frac{1}{\sqrt{2}} \\ 
\frac{1}{\sqrt{2}} & -\frac{1}{\sqrt{2}}
\end{pmatrix}
\begin{pmatrix}
\sqrt{\frac{2}{3}} & 0 \\ 
0 & \sqrt{\frac{1}{3}}
\end{pmatrix}
\begin{pmatrix}
\frac{1}{\sqrt{2}} & \frac{1}{\sqrt{2}} \\ 
\frac{1}{\sqrt{2}} & -\frac{1}{\sqrt{2}}
\end{pmatrix}
=
\frac{1}{2\sqrt{3}}
\begin{pmatrix}
\sqrt{2} + 1 & \sqrt{2} - 1 \\ 
\sqrt{2} - 1 & \sqrt{2} + 1
\end{pmatrix}
$$

In [24]:
sqrt_rho = sqrtm(rho_cvsum)
sqrt_rho

array([[0.69692343, 0.11957316],
       [0.11957316, 0.69692343]])

$$
\rho^{-1/2} = \frac{1}{\det \rho^{1/2}}
\frac{1}{2\sqrt{3}}
\begin{pmatrix}
\sqrt{2} + 1 & 1 - \sqrt{2}  \\ 
1 - \sqrt{2} & \sqrt{2} + 1
\end{pmatrix}
$$

In [25]:
inv_sqrt_rho = np.linalg.inv(sqrt_rho)
inv_sqrt_rho

array([[ 1.47839784, -0.25365297],
       [-0.25365297,  1.47839784]])

$$
\Pi_i = q_i \rho^{-1/2} \rho_i \rho^{-1/2}
$$

In [33]:
def pgm(q_list, rho_list):
    l = len(q_list)
    rho_cvsum = np.sum([q_list[i] * rho_list[i] for i in range(l)], axis=0)

    sqrt_rho = sqrtm(rho_cvsum)
    inv_sqrt_rho = np.linalg.inv(sqrt_rho)

    pgm_list = [q_list[i] * np.dot(np.dot(inv_sqrt_rho, rho_list[i]), inv_sqrt_rho) for i in range(l)]
    return pgm_list

In [42]:
pgm_list = pgm(q_list, rho_list)
pgm_list

[array([[ 0.72855339, -0.125     ],
        [-0.125     ,  0.02144661]]),
 array([[ 0.02144661, -0.125     ],
        [-0.125     ,  0.72855339]]),
 array([[0.25, 0.25],
        [0.25, 0.25]])]

$$
p_{i|i} = \text{Tr}[\Pi_i\rho_i]
$$

In [51]:
prob_list = [np.trace(np.dot(pgm_list[i], rho_list[i])) for i in range(3)]
prob_list

[0.7285533905932741, 0.7285533905932741, 0.5000000000000002]

$$
p_{error} = 1 - \sum_i q_ip_{i|i}
$$

In [50]:
error_prob = 1 - np.sum([q_list[i] * prob_list[i] for i in range(3)])
error_prob

0.34763107293781725